In [1]:
#*******************************************************************************************
 #
 #  File Name:  alphabet_soup_charity.ipynb
 #
 #  File Description:
 #      This interactive Python notebook, alphabet_soup_charity.ipynb, reads a csv file,
 #      charity_data.csv, in the folder, resources, and uses Python and deep learning
 #      methods to process the features in the provided dataset and create a binary classifier
 #      that can predict whether applicants for charity donations will be successful if
 #      funded by the nonprofit foundation, Alphabet Soup.
 #
 #      I reinstalled the scikit-learn module using the following commands to allow the
 #      RandomOverSampler function to work:
 #
 #      pip3 uninstall scikit-learn
 #      pip3 install scikit-learn==1.2.2
 #
 #
 #  Date            Description                             Programmer
 #  ----------      ------------------------------------    ------------------
 #  12/01/2023      Initial Development                     Nicholas J. George
 #
 #******************************************************************************************/

from google.colab import drive
drive.mount('/content/drive/')

import sys
sys.path.insert(0,'/content/drive/My Drive/alphabet_soup_charity_analysis')

!pip install -q dataframe_image
!pip install hvplot
!pip install silence-tensorflow
!pip install keras-tuner

import log_subroutines
import pandas_process_functions

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from silence_tensorflow import silence_tensorflow
silence_tensorflow()

import tensorflow as tf

Mounted at /content/drive/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 398.1/398.1 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for silence-tensorflow: filename=silence_tensorflow-1.2.1-py3-none-any.whl size=4465 sha256=bc1f340b462bf2f1e97f665f3039850edb1f9f19b17bbff4687afcea1ba540f9
  Stored in directory: /root/.cache/pip/wheels/7d/2c/24/e130d6102c0df56631b9db7479d9a6a53c5d97fb06b5f61b98
  Created wheel for support_developer: filename=support_developer-1.0.5-py3-none-any.whl size=5630 sha256=0c3bf3324e2b37579ae86c173b2fc3b9ee3bd70752133529df043c0132f1da01
  Stored in directory: /root/.cache/pip/wheels/b6/72/c8/3054a5897ba0713dfa7a941364d68cbd42b0755c8e2ec1c18c
Successfully built silence-tensorflow support_developer
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
CONSTANT_LOCAL_FILE_NAME = 'alphabet_soup_charity.ipynb'

CONSTANT_CHARITY_DATA_CSV_FILE_PATH = '/content/drive/My Drive/alphabet_soup_charity_analysis/resources/charity_data.csv'

CONSTANT_FIRST_OUTPUT_FILE_PATH = '/content/drive/My Drive/alphabet_soup_charity_analysis/resources/alphabet_soup_charity.keras'

# <br> **Section 1: Extraction**

## **1.1: Import and Read Charity Data**

In [3]:
data_type_dictionary \
    = {'EIN': str,
       'NAME':	str,
       'APPLICATION_TYPE': str,
       'AFFILIATION': str,
       'CLASSIFICATION': str,
       'USE_CASE': str,
       'ORGANIZATION': str,
       'STATUS': int,
       'INCOME_AMT': str,
       'SPECIAL_CONSIDERATIONS': str,
       'ASK_AMT': int,
       'IS_SUCCESSFUL': int}

charity_dataframe \
    = pd.read_csv(CONSTANT_CHARITY_DATA_CSV_FILE_PATH, dtype = data_type_dictionary)

log_subroutines.log_write_object(charity_dataframe)

## **1.2: Display Charity Data**

In [4]:
pandas_process_functions.return_formatted_table \
    (charity_dataframe, 'Table 1.2.1: Charity Data Table')

EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,"5,000",1
10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,"108,590",1
10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,"5,000",0
10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,"6,692",1
10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,"142,590",1
10556855,MINORITY ORGAN & TISSUE TRANSPLANT & EDUCATION PROGRAM OF TENNESSEE,T3,Independent,C1200,Preservation,Trust,1,0,N,"5,000",1
10558440,FRIENDS OF ARTS COUNCIL OF GREATER DENHAM SPRINGS INC,T3,Independent,C1000,Preservation,Trust,1,100000-499999,N,"31,452",1
10566033,ISRAEL EMERGENCY ALLIANCE,T3,Independent,C2000,Preservation,Trust,1,10M-50M,N,"7,508,025",1
10570430,ARAMCO BRATS INC,T7,Independent,C1000,ProductDev,Trust,1,1-9999,N,"94,389",1
10571689,INTERNATIONAL ASSOCIATION OF FIRE FIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,"5,000",0


# <br> **Section 2: Preprocessing**

## **2.1: Drop the Non-Beneficial ID Columns**

In [5]:
charity_dataframe.drop(['EIN','NAME'], axis = 1, inplace = True)

log_subroutines.log_write_object(charity_dataframe)

In [6]:
pandas_process_functions.return_formatted_table \
    (charity_dataframe, 'Table 1.2.2: Modified Charity Data Table')

APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
T10,Independent,C1000,ProductDev,Association,1,0,N,"5,000",1
T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,"108,590",1
T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,"5,000",0
T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,"6,692",1
T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,"142,590",1
T3,Independent,C1200,Preservation,Trust,1,0,N,"5,000",1
T3,Independent,C1000,Preservation,Trust,1,100000-499999,N,"31,452",1
T3,Independent,C2000,Preservation,Trust,1,10M-50M,N,"7,508,025",1
T7,Independent,C1000,ProductDev,Trust,1,1-9999,N,"94,389",1
T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,"5,000",0


## **2.2: Determine the Number of Unique Values in Each Column**

In [7]:
unique_value_count_each_column_series = charity_dataframe.nunique()

log_subroutines.log_write_object(unique_value_count_each_column_series)

In [8]:
log_subroutines.print_and_log_text(str(unique_value_count_each_column_series))

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


## **2.3: APPLICATION_TYPE Binning**

### **Look at APPLICATION_TYPE Value Counts for Binning**

In [9]:
application_type_value_counts_for_binning_series \
    = charity_dataframe['APPLICATION_TYPE'].value_counts().sort_values(ascending = False)

application_type_value_counts_for_binning_series.name = 'APPLICATION_TYPE'

log_subroutines.log_write_object(unique_value_count_each_column_series)

In [10]:
log_subroutines.print_and_log_text(str(application_type_value_counts_for_binning_series))

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


### **Choose a Cutoff Value and Create a List of Application Types to be Replaced**

In [11]:
application_types_to_replace \
    = list(application_type_value_counts_for_binning_series \
            [application_type_value_counts_for_binning_series < 500].index)

log_subroutines.log_write_object(application_types_to_replace)

In [12]:
log_subroutines.print_and_log_text(str(application_types_to_replace))

['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']


### **Replace Application Types in DataFrame**

In [13]:
for application_type in application_types_to_replace:

    charity_dataframe['APPLICATION_TYPE'] \
        = charity_dataframe['APPLICATION_TYPE'].replace(application_type, 'Other')

### **Check to Make Sure Binning was Successful**

In [14]:
application_type_value_counts_for_binning_series \
    = charity_dataframe['APPLICATION_TYPE'].value_counts().sort_values(ascending = False)

application_type_value_counts_for_binning_series.name = 'APPLICATION_TYPE'

log_subroutines.log_write_object(application_type_value_counts_for_binning_series)

In [15]:
log_subroutines.print_and_log_text(str(application_type_value_counts_for_binning_series))

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64


## **2.4: CLASSIFICATION Binning**

### **Look at CLASSIFICATION Value Counts for Binning**

In [16]:
classification_value_counts_for_binning_series \
    = charity_dataframe['CLASSIFICATION'].value_counts().sort_values(ascending = False)

classification_value_counts_for_binning_series.name = 'CLASSIFICATION'

log_subroutines.log_write_object(classification_value_counts_for_binning_series)

In [17]:
log_subroutines.print_and_log_text(str(classification_value_counts_for_binning_series))

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C5200        1
C2600        1
C4200        1
C2190        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


### **Look at CLASSIFICATION Value Counts > 1**

In [18]:
classification_value_counts_for_binning_series[classification_value_counts_for_binning_series > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1600        5
C1257        5
C1260        3
C0           3
C2710        3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

### **Choose a Cutoff Value and Create a List of Classifications to be Replaced**

In [19]:
classifications_to_replace \
    = list(classification_value_counts_for_binning_series \
            [classification_value_counts_for_binning_series < 100].index)

log_subroutines.log_write_object(classifications_to_replace)

### **Replace Classifications in DataFrame**

In [20]:
for classification in classifications_to_replace:

    charity_dataframe['CLASSIFICATION'] \
        = charity_dataframe['CLASSIFICATION'].replace(classification, 'Other')

### **Check to Make Sure Binning was Successful**

In [21]:
classification_value_counts_for_binning_series \
    = charity_dataframe['CLASSIFICATION'].value_counts().sort_values(ascending = False)

classification_value_counts_for_binning_series.name = 'CLASSIFICATION'

log_subroutines.log_write_object(classification_value_counts_for_binning_series)

In [22]:
log_subroutines.print_and_log_text(str(classification_value_counts_for_binning_series))

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64


## **2.5: Final Preprocessing**

### **Convert Categorical Data to Numeric with `pd.get_dummies`**

In [23]:
dummies_dataframe = pd.get_dummies(charity_dataframe)

log_subroutines.log_write_object(dummies_dataframe)

In [24]:
pandas_process_functions.return_formatted_table \
    (dummies_dataframe, 'Table 2.5.1: Dummies DataFrame Table')

STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C1270,CLASSIFICATION_C1700,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C2700,CLASSIFICATION_C3000,CLASSIFICATION_C4000,CLASSIFICATION_C5000,CLASSIFICATION_C7000,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
1,"5,000",1,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False,True,False
1,"108,590",1,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False
1,"5,000",0,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False,True,False
1,"6,692",1,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,True,False,False,False,False,False,False,True,False
1,"142,590",1,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,True,False
1,"5,000",1,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,True,False,False,False,False,False,False,False,False,True,False
1,"31,452",1,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,False,True,False
1,"7,508,025",1,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False
1,"94,389",1,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,True,False,False,False,False,False,False,False,True,False
1,"5,000",0,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False,True,False


### **Split Preprocessed Data into Features and Target Arrays**

In [25]:
y_nparray = dummies_dataframe['IS_SUCCESSFUL'].values

log_subroutines.log_write_object(y_nparray)

In [26]:
x_nparray = dummies_dataframe.drop(['IS_SUCCESSFUL'], axis = 1).values

log_subroutines.log_write_object(x_nparray)

### **Split the Preprocessed Data into a Training and Testing Dataset**

In [27]:
x_train_nparray, x_test_nparray, y_train_nparray, y_test_nparray \
    = train_test_split(x_nparray, y_nparray, random_state = 9)

In [28]:
log_subroutines.log_write_object(str(x_train_nparray) + '\n\n')

log_subroutines.log_write_object(str(x_test_nparray) + '\n\n')

log_subroutines.log_write_object(str(y_train_nparray) + '\n\n')

log_subroutines.log_write_object(str(y_test_nparray) + '\n\n')

### **Create a StandardScaler Instance**

In [29]:
current_standard_scalar = StandardScaler()

### **Fit the StandardScaler**

In [30]:
x_standard_scalar = current_standard_scalar.fit(x_train_nparray)

### **Scale the Data**

In [31]:
x_train_scaled_nparray = x_standard_scalar.transform(x_train_nparray)

log_subroutines.log_write_object(x_train_scaled_nparray)

In [32]:
x_test_scaled_nparray = x_standard_scalar.transform(x_test_nparray)

log_subroutines.log_write_object(x_test_scaled_nparray)

# <br> **Section 3: Compile, Train, Evaluate, and Export the Model**

## **3.1: Compile Model**

### **Model Definition**

In [33]:
neural_net_sequential = tf.keras.models.Sequential()

### **Layers**

In [34]:
input_dimension_integer = len(x_train_scaled_nparray[0])

# This line of code is the first layer.
neural_net_sequential.add \
    (tf.keras.layers.Dense(units = 95, activation = 'relu', input_dim = input_dimension_integer))

# This line of code is the hidden layer.
neural_net_sequential.add(tf.keras.layers.Dense(units = 38, activation = 'relu'))

# This line of code is the output layer.
neural_net_sequential.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

### **Check Model**

In [35]:
neural_net_sequential.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 95)                4750      
                                                                 
 dense_1 (Dense)             (None, 38)                3648      
                                                                 
 dense_2 (Dense)             (None, 1)                 39        
                                                                 
Total params: 8437 (32.96 KB)
Trainable params: 8437 (32.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### **Compile**

In [36]:
neural_net_sequential.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

## **3.2: Train Model**

In [37]:
neural_net_sequential.fit(x_train_scaled_nparray, y_train_nparray, epochs = 100)

Epoch 1/100
804/804 [==============================] - 10s 7ms/step - loss: 0.5684 - accuracy: 0.7215
Epoch 2/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5543 - accuracy: 0.7263
Epoch 3/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5512 - accuracy: 0.7299
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5500 - accuracy: 0.7297
Epoch 5/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5479 - accuracy: 0.7312
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5464 - accuracy: 0.7319
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5462 - accuracy: 0.7316
Epoch 8/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5446 - accuracy: 0.7323
Epoch 9/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5447 - accuracy: 0.7330
Epoch 10/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5443 - accur

## **3.3: Evaluate Model**

In [38]:
model_loss_float, model_accuracy_float \
    = neural_net_sequential.evaluate(x_test_scaled_nparray, y_test_nparray, verbose = 2)

log_subroutines \
    .print_and_log_text \
        (f'\nModel Loss: {round(model_loss_float * 100, 2)}%, '
         + f'Model Accuracy: {round(model_accuracy_float * 100, 2)}%')

268/268 - 1s - loss: 0.5593 - accuracy: 0.7315 - 564ms/epoch - 2ms/step

Model Loss: 55.93%, Model Accuracy: 73.15%


## **3.4: Export Model**

In [39]:
neural_net_sequential.save(CONSTANT_FIRST_OUTPUT_FILE_PATH)